# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [2]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

✅ LLM Client configured: Using 'gemini' with model 'gemini-2.5-pro'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [ ]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
# Your prompt here
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [ ]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
# Your prompt here
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [ ]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # TODO: Read the content of the schema file using load_artifact.
        schema_sql = ""
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        
        print("Tables created successfully.")

        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            # Load and execute the seed data
            pass # Your code here

        # TODO: Commit the changes to the database.
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        pass # Your code here

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.